# Avalanche Prediction via Machine Learning (so I begin...)

## Introduction
This is the first step of what I hope will be many in utilizing machine learning techniques to predict avalanche phenomena. In this notebook, I'm aim to present a little example of this by using a support vector machine (SVM) and three freely available datasets to predict avalanche detection (you'll see here soon why I wrote avalanche *detection* and not avalanche *occurrence*). Let's begin with the most important part, the data:.

## The datasets
For simplicities sake, I only looked at data from the current winter, Oct 1, 2019 to April 4, 2020.

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

start_time = pd.Timestamp("2019-10-01 00:00:00")
end_time = pd.Timestamp("2020-04-04 00:00:00")

### 1. Avalanches: 
The first dataset I got my hands on was from the Utah Avalanche Center. They have a [database of avalanches](https://utahavalanchecenter.org/avalanches) that goes all the way back to 1914. My mom's dad was 3 at the time. It's awesome. There's lots of great info in there. But alas, right off the bat we have a huge caveat to add to any conclusion that can be drawn with this data: this isn't a table of avalanches; it's a table of __*avalanche reportings*__. Thus, basically any model that uses this data is only going to be able to predict if somebody will __*see and report*__ an avalanche not that an avalanche occurs.

One idea for the future would be to use satellite imagery to detect avalanches. If the imagery is high res enough, a machine could be trained to detect them and provide coordinates. This would also provide the benefit of allowing both the slope angle and elevation to be calculated. Unfortunately, some days it's cloudy (NO WAY!), so this wouldn't provide comprehensive avalanche detection. Just spitballing.

For simplicity, I subsetted the data to only include avalanche reports from the "Salt Lake" region.

In [2]:
avy_data = pd.read_csv("avalanches.csv", parse_dates=['Date'])
avy_data = avy_data.rename(columns={'Date': 'date'})

# only Salt Lake
avy_data = avy_data.loc[avy_data.Region == 'Salt Lake']

# subset select time period
avy_data = avy_data[((avy_data.date >= start_time + pd.Timedelta(days=1)) &
                     (avy_data.date <= end_time))]

# count avalanches reported per day
avy_count = avy_data.date.value_counts()

# fill in missing datetimes
idx = pd.date_range(start_time + pd.Timedelta(days=1), end_time)
avy_count = avy_count.reindex(idx)
avy_count = avy_count.fillna(0)

# convert to any avalanche seen vs no avalanches seen
avy = avy_count.astype(bool).astype(int)
avy = avy.to_numpy()

### 2. Snow data:
SNOTEL has a pretty easy-to-use [web interface](http://wcc.sc.egov.usda.gov/reportGenerator/) to get information from their SNOTEL sites. For the current exploratory mission, I just used data from SITE 628 - "MILL-D NORTH". At this site, there are four measurements recorded: snow-water equivalent, snow depth, precipitation accumulation, and temperature. I didn't use precipitation accumulation because I don't really know what that means in this context (I haven't had time to figure out yet (April 17, 2020)). These measurements are taken hourly. There are a few lapses (I wanna say 300 bad recordings out of 13,000). I just interpolated to fill them in.

In [ ]:
snotel = pd.read_csv("snotel.csv", parse_dates=['date'])

# subset for select time period
snotel = snotel[(snotel.date > start_time + pd.Timedelta(hours=9)) &
                (snotel.date <= end_time + pd.Timedelta(hours=9))]

# drop pa_in cause I found some mismeasurements and what is this anyway?
snotel = snotel.drop(['pa_in'], axis=1)

# fill in missing dates
snotel = snotel.set_index(['date'])
idx = pd.date_range(snotel.index.min(), snotel.index.max(), freq='H')
snotel = snotel.reindex(idx)

# find nulls and interpolate
snotel = snotel.interpolate(axis=0)

### 3. Wind and Sky Cover data:
This data was surprisingly hard to find. I would think that there would be publicly available, hourly recordings of wind speed and direction recorded at elevations above 10,000 ft in the Wasatch but I couldn't find any (like at a resort's weather station?). I figured finding sky cover data would be more difficult. 

I found hourly recordings of both variables at a [NOAA website](https://www.ncdc.noaa.gov/cdo-web/). They were recorded at the Salt Lake Airport. The wind speed is just plain ole wind speed but the sky cover measurement is more abstruse. I'm gonna pull a tenured professor move and encourage the interested reader to read the [documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/NORMAL_HLY_documentation.pdf). I dropped all other remaining variables.

I'm not going to bore you by describing how I cleaned the data/dealt with it's eccentricities.

In [3]:
# convert feet or inches to inches
def conv_to_in(x):
    if x.endswith("'"):
        return float(x[:-1]) * 12
    if x.endswith('"'):
        return float(x[:-1])


# convert feet or inches to feet
def conv_to_ft(x):
    x = x.replace(',', '')
    if x.endswith("'"):
        return float(x[:-1])
    if x.endswith('"'):
        return float(x[:-1]) / 12


# parse sky_unparsed column
def parse_sky(x):
    if pd.isna(x):
        return x
    else:
        pieces = x.split()

        if len(pieces) == 1:
            return 0
        else:
            highest = pieces[-2]
            return int(highest[4:6])


noaa = pd.read_csv("noaa.csv", parse_dates=['DATE'])

# select features to keep and rename
noaa = noaa[['DATE', 'HourlySkyConditions', 'HourlyWindSpeed']]
noaa = noaa.rename(columns={'DATE': 'date',
                            'HourlySkyConditions': 'sky_unparsed',
                            'HourlyWindSpeed': 'windspeed'})

# subset for select time period
noaa = noaa[(noaa.date > start_time + pd.Timedelta(hours=9)) &
            (noaa.date <= end_time + pd.Timedelta(hours=9))]

# get rid of rows that don't make sense (are just a number)
noaa = noaa.loc[~(noaa['sky_unparsed'].apply(lambda x: str(x).isdigit()))]

# parse sky_unparsed column
noaa['sky'] = noaa['sky_unparsed'].apply(lambda x: parse_sky(x))
noaa = noaa.drop(['sky_unparsed'], axis=1)

# interpolate the missing data
noaa = noaa.interpolate(axis=0)

# round datetimes
noaa['date'] = noaa['date'].apply(lambda x: x.round('60min'))

# remove rows with duplicate datetimes
noaa = noaa.drop_duplicates('date')

# fill in missing datetimes
noaa = noaa.set_index(['date'])
idx = pd.date_range(noaa.index.min(), noaa.index.max(), freq='H')
noaa = noaa.reindex(idx)
noaa = noaa.interpolate(axis=0)

/home/john/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19,25,26,27,29,30,31,33,35,36,41,42,43,49,51,52,58,59,69,71,73,75,76,88,89,110) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
